# Practical 7: Waste Input-Output

Objectives:
- Understand the structure of a WIOT
- Build a WIO model in Python
- Analyze data from a WIO model

The methods used in this practical sessions come from:
Nakamura, S. and Kondo, Y. (2002), Input-Output Analysis of Waste Management. Journal of Industrial Ecology, 6: 39-63. https://doi.org/10.1162/108819802320971632

In [1]:
# Import packages
import pandas as pd
import numpy as np

# Set pandas number format
pd.options.display.float_format = '{:,.2f}'.format

## Exercise 1: 

### Conceptual understanding of WIO tables

|   |  E  | WT |  y  |
|---|-----|----|-----|
| E | 52  | 22 | 100 |
| WT| 349 | 44 | 50  |
  
<p align="center"> *Table 1: WIOT of a country* </p>

#### What are the expected units in the following parts of the simplified WIO system shown above? 

- E-WT quadrant (i.e., with value 22) 
  - **--> in Monetary flow**
- WT-WT quadrant (i.e., with value 44) 
  - **--> in Mass flow**
- and in Y (i.e., with values 100 and 50) 
  - **--> in Monetary flow**

#### Estimate the A-matrix. What do the coefficient represent in each quadrant? 



In [2]:
Z = np.array(
    [[ 52, 22],
     [349, 44]]
)

y = np.array([100, 50])

x = Z.sum(axis=1) + y  # x =  Ax + Y  =  Z + Y

A = Z / x.T  # would throw an error if divide by zero
A, x

(array([[0.29885057, 0.0496614 ],
        [2.00574713, 0.0993228 ]]),
 array([174, 443]))

## Exercise 2

### Construct a Waste Input-Output model

Use the data provided and the following figures to help you

- _Data adapted from source: https://www.f.waseda.jp/nakashin/WIO/wio2000sec103_0.06b_eng.xls_
- _For additional info: http://www.f.waseda.jp/nakashin/WIO.html_


![Waste Input-Output Accounting structure](../img/wio_1.gif)  
*Fig 1. Waste Input-Output Accounting structure (Nakamura and Kondo, 2009)*


#### Import data

In [3]:
FILEPATH = "../data/Waste_IO/Japan_WIO2000sec103.xlsx"

In [4]:
# Input-output table, including intermediate and final demand matrix
ZYeconomy = pd.read_excel(FILEPATH, sheet_name = 'ZYeconomy',  index_col = 0)

# Waste account, including intermediate and final demand matrix
ZYwaste = pd.read_excel(FILEPATH, sheet_name = 'ZYwaste',  index_col = 0)

# VA and satellite account
F = pd.read_excel(FILEPATH, sheet_name = 'F',  index_col = 0)

# Waste allocation matrix
Q = pd.read_excel(FILEPATH, sheet_name = 'Allocation',  index_col = 0)

# units
unit  = pd.read_excel(FILEPATH, sheet_name = 'Units')
unit = unit.set_index('name')

#### Get labels

|                 |   Economic sector    |    Waste treatment     |  **Y**             |
|-----------------|----------------------|------------------------|--------------------|
| Economic sector | **Z**<sub>E,E</sub>  |  **Z**<sub>E,WT</sub>  |  **Y**<sub>E</sub> |
| Waste treatment | **Z**<sub>WT,E</sub> |  **Z**<sub>WT,WT</sub> | **Y**<sub>WT</sub> |
| **r'**                                                        |||  **h**             |
|   Emissions     | Econ. Sec. emissions | Waste Treat. emissions |      Emissions     |  

*Table 2. Basic structure of a ZYeconomy table*

|                |   Economic sector     |    Waste treatment      |  **Y**              |
|----------------|-----------------------|-------------------------|---------------------|
| Waste inflows  | **Z**<sub>WI,E</sub>  |  **Z**<sub>WI,WT</sub>  |  **Y**<sub>WI</sub> |
| Waste outflows | **Z**<sub>WO,E</sub>  |  **Z**<sub>WO,WT</sub>  | **Y**<sub>WO</sub>  |


*Table 3. Basic structure of a ZYwaste table*

|                 |     Waste flows       |
|-----------------|-----------------------|
| Waste treatment |       **Q**           |


*Table 4. Basic structure of a Allocation table*

In [5]:
economic_sector_labels = ZYeconomy.index  
waste_sector_labels = Q.index    
waste_flow_labels = Q.columns    
all_sector_labels = economic_sector_labels.append(waste_sector_labels)
Y_labels = ZYeconomy.drop(columns=all_sector_labels).columns

# Get labels for waste inflows / outflows (= Net Flows = Flows out - Flows in)
MASK = ZYwaste.index.str.contains("Wo \(")
waste_out_labels = ZYwaste.loc[MASK].index

MASK = ZYwaste.index.str.contains('Wi \(')
waste_in_labels = ZYwaste.loc[MASK].index

#### Create WIOT table 

1. Compute net waste flows ($\mathbf{Z}_{\text{WF}} = \mathbf{Z}_{\text{WO}} - \mathbf{Z}_{\text{WI}}$)
2. Convert waste flows into waste treatment ($\mathbf{Z}_{\text{WT}} = \mathbf{QZ}_{\text{WF}}$)


|                 |   Economic sector                                  |    Waste treatment     |  **Y**             |
|-----------------|----------------------------------------------------|------------------------|--------------------|
| Economic sector | **Z**<sub>E,E</sub>                                |  **Z**<sub>E,WT</sub>  |  **Y**<sub>E</sub> |
| Waste treatment | **Q**(**Z**<sub>WO,E</sub> - **Z**<sub>WI,E</sub>) |  **Q**(**Z**<sub>WO,WT</sub> - **Z**<sub>WI,WT</sub>) |**Q**(**Y**<sub>WO</sub> - **Y**<sub>WI</sub>) |
  
*Table 5. Basic structure of a WIOT table*

In [6]:
# Reorder labels to make sure to compute flows in the same order
waste_flow_labels = waste_flow_labels.sort_values()
waste_out_labels = waste_out_labels.sort_values()
waste_in_labels = waste_in_labels.sort_values()

# Create empty Dataframes to store calculation
ZY_wf = pd.DataFrame(
    data=np.nan, 
    index=waste_flow_labels, 
    columns=ZYeconomy.columns,
)

# Use loc to ensure to process rows in the same order
ZY_wf.loc[waste_flow_labels] = ZYwaste.loc[waste_out_labels].values - ZYwaste.loc[waste_in_labels].values

# Sanity Check
if ZY_wf.isna().any().any():
    raise ValueError('Missing net flows (nan values are present)')

In [7]:
ZY_e = ZYeconomy.loc[economic_sector_labels, :]
QZY_wf = Q @ ZY_wf

WIOT = pd.concat([ZY_e, QZY_wf], axis=0)

In [8]:
Z_wiot = WIOT.loc[:, all_sector_labels]

# Visualize result
Z_wiot

,Crop cultivation,Livestock,Agricultural services,Forestry,Fisheries,Metallic ores,Non-metallic ores,Coal mining,Crude petroleum and natural gas,Foods,...,Landfill,Shredding: bulky textile,Shredding: wooden furniture,"Shredding: bikes, ovens",Shredding: small electric appliances,Shredding: TV sets,Shredding: refrigerators,Shredding: washing machines,Shredding: air conditioners,Shredding: automobiles
Crop cultivation,"165,202.00","277,272.00","3,609.00","1,362.00",0.00,0.00,0.00,0.00,0.00,"3,491,681.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Livestock,"47,293.00","268,281.00","6,749.00","2,536.00",0.00,0.00,0.00,0.00,0.00,"1,974,122.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Agricultural services,"341,459.00","157,409.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Forestry,"2,325.00",0.00,0.00,"190,232.00","1,172.00",106.00,89.00,309.00,19.00,"17,367.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Fisheries,0.00,0.00,0.00,0.00,"93,568.00",0.00,0.00,0.00,0.00,"1,262,796.00",...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Shredding: TV sets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: refrigerators,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: washing machines,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: air conditioners,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [9]:
Y_wiot = WIOT.loc[:, Y_labels]

# Visualize result
Y_wiot

,Consumption expenditure outside households (column),Consumption expenditure of households,Consumption expenditure of private non-profit institutions serving households,Consumption expenditure of general government,Consumption expenditure of general government (social fixed capital depreciation),Gross domestic fixed capital formation (public),Gross domestic fixed capital formation (private),Increase in stocks,Exports,Exports (direct purchase),Balancing sector
Crop cultivation,"66,019.00","3,001,038.00",0.00,0.00,0.00,0.00,"44,191.00","19,150.00","11,809.00",83.00,562.00
Livestock,0.00,"209,839.00",0.00,0.00,0.00,0.00,"149,290.00","38,327.00",824.00,11.00,40.00
Agricultural services,0.00,"84,720.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Forestry,"4,179.00","187,005.00",0.00,0.00,0.00,0.00,0.00,"679,130.00","1,547.00",1.00,61.00
Fisheries,"21,023.00","392,104.00",0.00,0.00,0.00,0.00,0.00,"37,110.00","54,924.00",47.00,"2,109.00"
...,...,...,...,...,...,...,...,...,...,...,...
Shredding: TV sets,0.00,"226,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: refrigerators,0.00,"240,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: washing machines,0.00,"113,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: air conditioners,0.00,"154,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Exercise 3

### From a consumption-based perspective, which are the major contributors to landfill area footprint?

#### Calculate Leontief inverse

In [10]:
# Compute x
x = WIOT.sum(axis=1)  # x  =  Ax + Y  =  Z + Y

# Compute A
A = Z_wiot / x.T
A = A.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# Create an identity matrix the same order
Id = np.identity(len(all_sector_labels))

# Compute leontief inverse matrix in the Demand-pull model
L_values = np.linalg.inv((Id - A))
L = pd.DataFrame(
    data=L_values,
    index=all_sector_labels, 
    columns=all_sector_labels
)

#### Calculate Landfill intensity

In [11]:
# compute extension intensity 
f = F.loc[:, all_sector_labels] / x.T  # Don't keep the Y part
f = f.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

#### Calculate landfill footprint per economic sector (LF)

In [12]:
# select Landfill extension indicator
MASK = f.index.str.contains("Landfill")
LANDFILL = f.loc[MASK].index
LANDFILL

Index(['Landfill area', 'Landfill volume'], dtype='object', name='category')

In [13]:
f_landfill_area = f.loc['Landfill area']
fLdiagY_landfill_area = (f_landfill_area @ L) * Y_wiot.T
fLdiagY_landfill_area = fLdiagY_landfill_area.T
fLdiagY_landfill_area

,Consumption expenditure outside households (column),Consumption expenditure of households,Consumption expenditure of private non-profit institutions serving households,Consumption expenditure of general government,Consumption expenditure of general government (social fixed capital depreciation),Gross domestic fixed capital formation (public),Gross domestic fixed capital formation (private),Increase in stocks,Exports,Exports (direct purchase),Balancing sector
Crop cultivation,"-13,428.91","-610,440.54",-0.00,-0.00,-0.00,-0.00,"-8,988.88","-3,895.30","-2,402.07",-16.88,-114.32
Livestock,-0.00,"-319,766.09",-0.00,-0.00,-0.00,-0.00,"-227,497.65","-58,405.13","-1,255.66",-16.76,-60.95
Agricultural services,-0.00,"-38,171.99",-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
Forestry,224.09,"10,027.96",0.00,0.00,0.00,0.00,0.00,"36,417.67",82.96,0.05,3.27
Fisheries,"-2,718.58","-50,704.83",-0.00,-0.00,-0.00,-0.00,-0.00,"-4,798.87","-7,102.48",-6.08,-272.72
...,...,...,...,...,...,...,...,...,...,...,...
Shredding: TV sets,0.00,"624,699.85",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: refrigerators,0.00,"659,756.70",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: washing machines,0.00,"312,349.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Shredding: air conditioners,0.00,"421,432.16",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [14]:
fLdiagY_landfill_area.sum(axis=1).nlargest(5)

# Uncomment to check unit of Landfill area
# unit.loc['Landfill area']

Landfill                                   74,171,323.30
Building construction                      67,852,852.48
Water supply                               14,937,332.58
Shredding: automobiles                     14,877,076.50
Other civil engineering and construction   10,038,927.53
dtype: float64

In [15]:
# similarly, the landfill footprint expressed in terms of Volume
f_landfill_volume = f.loc['Landfill volume']
fLdiagY_landfill_volume = (f_landfill_volume @ L) * Y_wiot.T
fLdiagY_landfill_volume = fLdiagY_landfill_volume.T
fLdiagY_landfill_volume.sum(axis=1).nlargest(5)

# Uncomment to check unit of Landfill area
# unit.loc['Landfill volume']

Landfill                                   32,862,100.76
Building construction                      30,062,660.02
Water supply                                6,618,085.08
Shredding: automobiles                      6,591,388.22
Other civil engineering and construction    4,447,813.97
dtype: float64

### Credits

First version of the exercise developed by dr. J.F.D. Rodrigues, improvements by dr. G.A. Aguilar Hernandez (2022) and F. Donati (2023)  